## Example Running PROSAIL algorithm over a UAV imagery
October 2025
By Hector Nieto, Alfonso Torres

#necessary information
1. multispectral imagery with no thermal band and scaled to a pixel size that addresses field LAI measurement scales (several meters)
2. sensor response function of the UAV sensor (micasense.txt)
3. Install python libraries as requested in README.md
4. Make sure GDAL is adequately installed.

Using MicaSense cameras nomenclature:
https://support.micasense.com/hc/en-us/articles/214878778-What-is-the-center-wavelength-and-bandwidth-of-each-filter-for-MicaSense-sensors

For MicaSense Dual-Camera use:

BANDS = ["B06", "B01", "B07", "B02", "B08","B03", "B09", "B05", "B10", "B04"]

For MicaSense Altum Camera use:

BANDS = [ "B01", "B02","B03", "B05", "B04"]


In [ ]:
## Running ProSAIL simulations on a MicaSense 10 band sensor
layer_path = r"D:\Github\pypro4sail\inputs\20220708_almond_ripperdan_fl7_1329_10band_refl.tif"

# scaling factor reflectance image
layer_scale = 1

date_flight = "20220708" # date of the flight YYYYMMDD
time_flight = "12:29" # time of the flight HH:MM (standard time)

# output folder
output_folder = r"D:\Github\pypro4sail\outputs"
output_name = "micasense_10bands"

# Micasense bands in the 10 band sensor (in order of wavelength)
BANDS = ["B06", "B01", "B07", "B02", "B08","B03", "B09", "B05", "B10", "B04"]



In [17]:
# import necessary libraries
from pathlib import Path
import multiprocessing as mp
import numpy as np
import pandas as pd
import datetime as dt
from datetime import date
from datetime import datetime
from osgeo import gdal
import Py6S as sixs
from pyproj import Proj #added
from pypro4sail import machine_learning_regression as inv
from pyTSEB import meteo_utils as met
import sklearn 
from sklearn.ensemble import RandomForestRegressor as rf_sklearn
import logging

log = logging.getLogger(__name__)

In [18]:
# Path to the pyPro4SAIL soil library and SRF library
__file__ = globals().get("pypro4sail", None) 

SOIL_LIBRARY = Path(inv.__file__).parent / "spectra" / "soil_spectral_library"
SRF_LIBRARY = Path(inv.__file__).parent / "spectra" / "sensor_response_functions"

DEFAULT_AOD = 0.1
DEFAULT_WVP = 1.0

# Pro4SAIL object parameter names
OBJ_PARAM_NAMES = ["Cab", "Car", "Cm", "Cw", "Ant", "Cbrown",
                   "LAI", "leaf_angle"]

In [19]:
# Function to run 6S and get diffuse radiation fraction
def get_diffuse_radiation_6S(aot, wvp, sza, saa, date,
                             altitude=0.1, wls_step=10, n_jobs=None):

    s = sixs.SixS()
    s.atmos_profile = sixs.AtmosProfile.PredefinedType(
                            sixs.AtmosProfile.MidlatitudeSummer)

    s.aeroprofile = sixs.AeroProfile.PredefinedType(
                            sixs.AeroProfile.Continental)
    
    s.ground_reflectance = sixs.GroundReflectance.HomogeneousLambertian(0)

    if np.isfinite(wvp) and wvp > 0:
        s.atmos_profile = sixs.AtmosProfile.UserWaterAndOzone(wvp, 0.9)
    
    if np.isfinite(aot) and aot > 0:
        s.aot550 = aot

    s.geometry.solar_z = sza
    s.geometry.solar_a = saa
    s.geometry.view_z = 0
    s.geometry.view_a = 0
    s.geometry.day = date.day
    s.geometry.month = date.month

    s.altitudes.set_target_custom_altitude(altitude)
    s.wavelength = sixs.Wavelength(0.4, 2.5)

    wls = np.arange(400, 2501)
    wls_sim = np.arange(400, 2501, wls_step)

    wv, res = sixs.SixSHelpers.Wavelengths.run_wavelengths(s,
                                                           wls_sim / 1000.,
                                                           verbose=False,
                                                           n=n_jobs)
    
    eg_d = np.array(sixs.SixSHelpers.Wavelengths.extract_output(res, 
                                                    'diffuse_solar_irradiance'))
    
    eg_s = np.array(sixs.SixSHelpers.Wavelengths.extract_output(res, 
                                                    'direct_solar_irradiance'))
    
    eg_d = np.maximum(eg_d, 0)
    eg_s = np.maximum(eg_s, 0)
    skyl = np.full_like(wls, np.nan, dtype=np.float64)
    # Fill the diffuse values into a full wavelenght array
    valid = np.in1d(wls, wls_sim, assume_unique=True)
    skyl[valid] = eg_d / (eg_d + eg_s)
    # Fill nans by linear interpolation
    nans, x = np.isnan(skyl), lambda z: z.nonzero()[0]
    skyl[nans] = np.interp(x(nans), x(~nans), skyl[~nans])    
    
    return skyl

In [20]:
# Function to build soil database for Pro4SAIL simulations
def build_soil_database(soil_albedo_factor,
                        soil_library=SOIL_LIBRARY):

    soil_library = Path(soil_library)
    n_simulations = np.size(soil_albedo_factor)
    soil_files = list(soil_library.glob('jhu.*spectrum.txt'))
    n_soils = len(soil_files)
    soil_spectrum = []
    for soil_file in soil_files:
        r = np.genfromtxt(soil_file)
        soil_spectrum.append(r[:, 1])

    multiplier = int(np.ceil(float(n_simulations / n_soils)))
    soil_spectrum = np.asarray(soil_spectrum * multiplier)
    soil_spectrum = soil_spectrum[:n_simulations]
    soil_spectrum = soil_spectrum * soil_albedo_factor.reshape(-1, 1)
    soil_spectrum = np.clip(soil_spectrum, 0, 1)
    soil_spectrum = soil_spectrum.T
    return soil_spectrum

In [21]:
# Helper to read a GDAL image file
def get_raster_data(input_file_path, bands=None):
    """
    Helper to read a GDAL image file

    Parameters
    ----------
    input_file_path : str of Path object
        Path to the input GDAL image file

    Returns
    -------
    array : 2D array
        Output numpy array
    """
    fid = gdal.Open(str(input_file_path), gdal.GA_ReadOnly)
    if isinstance(bands, type(None)):
        bands = range(1, fid.RasterCount + 1)
    array = []
    for band in bands:
        array.append(fid.GetRasterBand(band).ReadAsArray())
    del fid
    array = np.squeeze(np.array(array))
    return array

In [22]:
# Helper to get raster information
def raster_info(input_file_path):

    fid = gdal.Open(str(input_file_path), gdal.GA_ReadOnly)
    gt = fid.GetGeoTransform()
    proj = fid.GetProjection()
    x_size = fid.RasterXSize
    y_size = fid.RasterYSize
    bands = fid.RasterCount
    lr_x = gt[0] + x_size * gt[1] + y_size * gt[2]
    lr_y = gt[3] + x_size * gt[4] + y_size * gt[5]
    extent = [gt[0], lr_y, lr_x, gt[3]]
    del fid
    center = np.mean([gt[0], lr_x]), np.mean([gt[3], lr_y])
    p = Proj(proj)
    center_geo = p(center[0], center[1], inverse=True)

    return proj, gt, x_size, y_size, extent, center_geo, bands

In [23]:
# Helper to save a GDAL Cloud Optimized GeoTiff
def save_image(in_array, gt, proj, output_file, no_data_value=np.nan):
    """
    Helper to save a GDAL Cloud Optimized GeoTiff

    Parameters
    ----------
    in_array : 2D array
        Numpy image array
    gt : list of floats
        Output GDAL geotransform
    proj : str
        Output GDAL projection
    output_file : str or Path object
        Path to the output GeoTiff image
    """
    memDriver = gdal.GetDriverByName("MEM")
    if in_array.dtype == bool:
        gdal_type = gdal.GDT_Byte
        no_data_value = None
    elif in_array.dtype == int:
        gdal_type = gdal.GDT_Int32
    else:
        gdal_type = gdal.GDT_Float32

    shape = in_array.shape
    if len(shape) > 2:
        ds = memDriver.Create("MEM", shape[1], shape[0], shape[2], gdal_type)
        ds.SetProjection(proj)
        ds.SetGeoTransform(gt)
        for i in range(shape[2]):
            ds.GetRasterBand(i+1).WriteArray(in_array[:, :, i])

    else:
        ds = memDriver.Create("MEM", shape[1], shape[0], 1, gdal_type)
        ds.SetProjection(proj)
        ds.SetGeoTransform(gt)
        ds.GetRasterBand(1).WriteArray(in_array)
    ds.FlushCache()
    if output_file == "MEM":
        ds.GetRasterBand(1).SetNoDataValue(no_data_value)
        out_ds = ds
    else:
        driver_opt = ['COMPRESS=DEFLATE',
                      'PREDICTOR=YES',
                      'BIGTIFF=IF_SAFER',
                      'NUM_THREADS=ALL_CPUS']
        
        out_ds = gdal.Translate(str(output_file), 
                                ds, 
                                format="COG", 
                                creationOptions=driver_opt,
                                noData=no_data_value, 
                                stats=False)
    # If GDAL driers for other formats do not exist then default to GeoTiff
    if out_ds is None:
        log.warning("Selected GDAL driver is not supported!"
                    "Saving as GeoTiff!")
        driver_opt = ['COMPRESS=DEFLATE',
                      'PREDICTOR=1',
                      'BIGTIFF=IF_SAFER',
                      'NUM_THREADS=ALL_CPUS']

        ds = gdal.Translate(str(output_file), 
                            ds, 
                            format="GTiff", 
                            creationOptions=driver_opt,
                            noData=no_data_value, 
                            stats=True)
        if ds is None:
            raise IOError(f"Image {output_file} could not be created")
    else:
        ds = out_ds

    return ds   

In [24]:
# Main function to run biophysical retrieval from Sentinel-2 L2A FORCE products
def force_biophysical(boa_file,
                      date_obj,
                      output_folder,
                      output_name,
                      sza,
                      saa,
                      vza=0,
                      aot=DEFAULT_AOD,
                      wvp=DEFAULT_WVP,
                      n_simulations=40000,
                      n_jobs=-1,
                      srf_file= SRF_LIBRARY / f'micasense.txt'):
    """Creates biophysical traits from Sentinel-2 L2A FORCE

    Parameters
    ----------
    boa_file : str or Path object
        Path to the BOA l2a file generated by FORCE
    date_obj : datetime object
        Date object representing the acquisition date
    sza : float
        Solar zenith angle in degrees
    output_folder : str or Path object
        Path to the folder where the biophysical products will be saved
    n_simulations : int, optional
        Number of PROSPECT-D + 4AIL simulations
    n_jobs : int, optional
        Number of CPUs used during parallel processing

    Returns
    -------
    None
    """
    start_time = dt.datetime.today()
    #scene_name = boa_file.stem
    params_orig = inv.build_prosail_database(n_simulations,
                                             distribution=inv.SALTELLI_DIST)

    scikit_regressor_opts = {"n_estimators": 100,
                             "min_samples_leaf": 1,
                             "n_jobs": n_jobs}

    
#    date_obj = dt.datetime.strptime(date, "%Y%m%d")
    prj, gt, _, _, _, center, *_ = raster_info(boa_file)


    # Get Day of the Year
    doy = int(date_obj.strftime("%j"))

    log.info(f"Running 6S for estimation of diffuse/direct irradiance")
    skyl = get_diffuse_radiation_6S(aot, wvp, sza, saa, date_obj,
                                    altitude=0.1)

    # Stack spectral bands
    srf = []
    
    srfs = np.genfromtxt(srf_file, dtype=None, names=True)
    for band in BANDS:
        srf.append(srfs[band])

    log.info(f"Builing standard soil database")
    soil_spectrum = build_soil_database(params_orig["bs"])
    log.info(f"Building {np.size(params_orig['bs'])}" 
             f"PROSPECTD+4SAIL simulations")
    if "fAPAR" in OBJ_PARAM_NAMES or "fIPAR" in OBJ_PARAM_NAMES:
        calc_fapar = True
    else:
        calc_fapar = False
    if n_jobs <= 1:
        rho_canopy_vec, params = inv.simulate_prosail_lut(params_orig,
                                                          np.arange(400,2501),
                                                          soil_spectrum,
                                                          skyl=skyl,
                                                          sza=sza,
                                                          vza=vza,
                                                          psi=0,
                                                          srf=srf,
                                                          outfile=None,
                                                          calc_FAPAR=calc_fapar,
                                                          reduce_4sail=True)

    else:
        rho_canopy_vec, params = inv.simulate_prosail_lut_parallel(
            n_jobs,
            params_orig,
            np.arange(400,2501),
            soil_spectrum,
            skyl=skyl,
            sza=sza,
            vza=vza,
            psi=0,
            srf=srf,
            outfile=None,
            calc_FAPAR=calc_fapar,
            reduce_4sail=True)

    log.info(f"Training Random forest for {','.join(OBJ_PARAM_NAMES)}")
    params = pd.DataFrame(params)
    reg = rf_sklearn(**scikit_regressor_opts)
    # Apply model to S2 image
    image_array = get_raster_data(boa_file) 
    dims = image_array.shape[1:]
    #print(dims)
    image_array = image_array.reshape((image_array.shape[0], -1)).T
    image_array = image_array / layer_scale
    
    for i, param in enumerate(OBJ_PARAM_NAMES):
        log.info(f'Applying {param} regression model to image')
        reg = reg.fit(rho_canopy_vec, params[param])
        output = reg.predict(image_array)
        output = output.reshape(dims)
        if param == 'fAPAR' or param == 'fIPAR':
            min_value = 0
            max_value = 1
        else:
            min_value = inv.prosail_bounds[param][0]
            max_value = inv.prosail_bounds[param][1]

        output = np.clip(output, min_value, max_value)
        output_file = Path(output_folder) /  f"{output_name}_{param.upper()}.tif"
        log.info(f"Saving {param} in {output_file}")
        save_image(output, gt, prj, output_file)
        del output

    del reg, rho_canopy_vec, params
    """
    # Narrowband to Broadband reflectance
    platform = f'Sentinel{satellite[-2:]}'
    rho_par = np.full(dims, np.nan)
    rho_nir = np.full(dims, np.nan)
    valid = valid.reshape(dims)
    rho_par[valid], rho_nir[valid] = broadband_reflectance(image_array.T,
                                                           platform)[:2]

    output_file = output_folder / f"{scene}_RHO-PAR.tif"
    save_image(rho_par, gt, prj, output_file)
    del rho_par
    output_file = output_folder / f"{scene}_RHO-NIR.tif"
    save_image(rho_nir, gt, prj, output_file)
    del rho_nir
    """

    del image_array
    elapsed = (dt.datetime.today() - start_time).total_seconds()
    log.info(f"Finished in {elapsed/60:.1f} minutes")

In [25]:
# Get raster information
layer_info = raster_info(layer_path)

# Get standard longitude
center_geo = layer_info[-2]
stdlon = round(center_geo[0]/15,0)*15


# Convert string to date
date_object = datetime.strptime(date_flight, "%Y%d%m").date()
doy = int(date_object.strftime("%j"))

# Convert time string to decimal time
time_object = datetime.strptime(time_flight, "%H:%M").time()
dec_time = time_object.hour + time_object.minute / 60.0

# Get Solar angles
sza, saa = met.calc_sun_angles(center_geo[1],
                                center_geo[0], 
                                stdlon,
                                doy, 
                                dec_time)

sza =float(sza)
saa =float(saa)

In [26]:
# Run biophysical retrieval for drone image
force_biophysical(layer_path,
                      date_object,
                      output_folder,
                      output_name,
                      sza,
                      saa,
                      vza=0,
                      aot=DEFAULT_AOD,
                      wvp=DEFAULT_WVP,
                      n_simulations=1000, #for full run, this value should be 40000
                      n_jobs=-1,
                      srf_file= SRF_LIBRARY / f'micasense.txt')

c:\Users\alfon\.conda\envs\pyTSEB\Lib\site-packages\SALib\sample\sobol.py:136: UserWarning: The balance properties of Sobol' points require n to be a power of 2.
  base_sequence = qrng.random(N)


Running for many wavelengths - this may take a long time


C:\Users\alfon\AppData\Local\Temp\ipykernel_13512\3456341618.py:48: DeprecationWarning: `in1d` is deprecated. Use `np.isin` instead.
  valid = np.in1d(wls, wls_sim, assume_unique=True)


Starting 1008 Simulations
